In [1]:
import time
import pygame
import tkinter as tk
from tkinter import font as tkFont
from PIL import Image, ImageTk
import random
import serial
import pywitmotion as wit
import time
import threading
import csv
from datetime import datetime

pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [34]:
import serial.tools.list_ports

# 사용 가능한 시리얼 포트 목록 가져오기
available_ports = serial.tools.list_ports.comports()

if available_ports:
    print("사용 가능한 포트 목록:")
    for port in available_ports:
        print(port.device)  # 포트 이름 출력
else:
    print("사용 가능한 포트가 없습니다.")


사용 가능한 포트 목록:
/dev/cu.wlan-debug
/dev/cu.Bluetooth-Incoming-Port
/dev/cu.HC-06


In [32]:
on_serial_communication=True
connected = False
#port = 'COM3'
port = '/dev/cu.HC-06'
baud = 115400
t=5
csv_data = []
decode_q=[]
msgs_num = 0

# 비프음과 텍스트 순서대로 정의
sound_texts = [
    "앞니 바깥면으로 이동해주세요.",
    "앞니 바깥면",
    "왼쪽 바깥면",
    "오른쪽 바깥면",
    "왼쪽 위 씹는면",
    "왼쪽 아래 씹는면",
    "오른쪽 위 씹는면",
    "오른쪽 아래 씹는면",
    "앞니 위 안쪽면",
    "앞니 아래 안쪽면",
    "왼쪽 위 안쪽면",
    "왼쪽 아래 안쪽면",
    "오른쪽 위 안쪽면",
    "오른쪽 아래 안쪽면",
    "종료"
]

sound_texts2 = [
    "자유롭게 양치해주세요",
    "종료"
]

# Pygame 초기화
pygame.mixer.init()

# Set the background color to #292E3B
root = tk.Tk()
root.configure(bg="#F0F0F0")
root.title("2021013218 김도윤")

main_bg="#F0F0F0"

# Load the custom font
font_title = tkFont.Font(family="NEXON Football Gothic B", size=70)

# 프레임 생성
title_frame = tk.Frame(root, bg=main_bg)
title_frame.pack(side="top")

# 첫 번째 레이블 - "T"
title_t_label = tk.Label(title_frame, text="\nT", font=font_title, anchor="center", bg=main_bg, fg="#A89755")
title_t_label.pack(side="left")

# 두 번째 레이블 - "romatz"
title_label = tk.Label(title_frame, text="\nromatz", font=font_title, anchor="center", bg=main_bg, fg="black")
title_label.pack(side="left")

# Load the custom font
font_subtitle = tkFont.Font(family="NEXON Lv1 Gothic", size=30)

# 텍스트 표시 레이블
subtitle_label = tk.Label(root, text="권민욱 임대연 김도윤", font=font_subtitle, anchor="center", bg=main_bg, fg="#807F83")
subtitle_label.pack(side="top", fill="both", expand=True)

# Load the custom font
font_15size = tkFont.Font(family="NEXON Football Gothic B", size=15)
font_30size = tkFont.Font(family="NEXON Football Gothic B", size=30)
font_70size = tkFont.Font(family="NEXON Football Gothic B", size=70)

# 텍스트 표시 레이블
press_kr_label = tk.Label(root, text="\n시작하려면 스페이스키를 누르세요", font=font_15size, anchor="center", bg=main_bg, fg="#479C2F")
press_kr_label.pack(side="top", fill="both", expand=True)

# 텍스트 표시 레이블
text_label = tk.Label(root, text="PRESS SPACE KEY", font=font_30size, anchor="center", bg=main_bg, fg="#479C2F")
text_label.pack(side="top", fill="both", expand=True)

# "다음구역:" 레이블
font_next = tkFont.Font(family="NEXON Football Gothic B", size=30)
next_label = tk.Label(root, text="", font=font_next, foreground="gray", bg=main_bg)

# "남은 시간:" 레이블
font_timer = tkFont.Font(family="NEXON Football Gothic B", size=50)
timer_label = tk.Label(root, text="", font=font_timer, anchor="center", bg=main_bg, fg="white")

# 창 크기 설정
root.geometry("800x400")
root.update_idletasks()
width = root.winfo_width()
height = root.winfo_height()
x = (root.winfo_screenwidth() // 2) - (width // 2)
y = (root.winfo_screenheight() // 2) - (height // 2)
root.geometry('{}x{}+{}+{}'.format(width, height, x, y))

current_text_index = 0  # 현재 텍스트의 인덱스
timer_running = False
blinking = True
is_c=False
is_space=False
msg=[]
msg_label=[]
msg_time=[]
label=None

def extract_values(s):
    values = s.strip('[]').split('],[')
    values = [val.split() for val in values]
    return [item.strip() for sublist in values for item in sublist]

def save_csv(csv_data):
    # 현재 날짜 및 시간 가져오기
    now = datetime.now()
    formatted_date_time = now.strftime("%Y%m%d_%H%M%S")

    # CSV 파일에 데이터 쓰기
    file_name = f"{formatted_date_time}.csv"
    with open(file_name, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['time', 'ax', 'ay', 'az', 'gyro_x', 'gyro_y', 'gyro_z', 'label'])  # 헤더 작성
        writer.writerows(csv_data)

def save_values():
    global csv_data
    idx=0
    while idx+1<len(msg):
        # q = wit.get_quaternion(s)
        # q = wit.get_magnetic(msg)
        # q = wit.get_angle(msg)

        q1 = wit.get_acceleration(msg[idx])
        q2 = wit.get_gyro(msg[idx+1])
        msg_idx=msg_label[idx]
        msg_time_idx=msg_time[idx]
        
        if (q1 is not None)and(q2 is not None):
            decode_q.append(f'{msg_time_idx},{q1},{q2},{msg_idx}')
        
        idx+=1
    print(decode_q)

    # Extracting and restructuring the data
    for item in decode_q:
        values = extract_values(item)
        c_time, ax, ay, az, gyro_x, gyro_y, gyro_z, label = values
        csv_data.append([c_time, ax, ay, az, gyro_x, gyro_y, gyro_z, label])

    save_csv(csv_data)

def serial_communication():
    global msg, msg_label, msg_time
    if on_serial_communication==True:
        with serial.Serial(port, baud, timeout=5) as ser:
            s = ser.read()
            while True:
                s = ser.read_until(b'U')
                current_time = datetime.now().strftime("%H:%M:%S.%f")
                msg.append(s)
                msg_label.append(f'[{label}]')
                msg_time.append(f'[{current_time}]')

serial_thread = threading.Thread(target=serial_communication)
serial_thread.start()

def toggle_text_label():
    if blinking:
        current_color = text_label['fg']
        target_color = "#479C2F" if current_color == main_bg else main_bg

        r1, g1, b1 = int(current_color[1:3], 16), int(current_color[3:5], 16), int(current_color[5:7], 16)
        r2, g2, b2 = int(target_color[1:3], 16), int(target_color[3:5], 16), int(target_color[5:7], 16)

        r_step = (r2 - r1) / 200  # 10 단계로 나누어서 점진적으로 변경
        g_step = (g2 - g1) / 200
        b_step = (b2 - b1) / 200

        def update_color(step):
            if blinking:
                if step <= 200:
                    r = int(r1 + r_step * step)
                    g = int(g1 + g_step * step)
                    b = int(b1 + b_step * step)
                    color = f'#{r:02X}{g:02X}{b:02X}'
                    text_label.config(fg=color)
                    press_kr_label.config(fg=color)
                    root.after(5, lambda: update_color(step + 1))
                else:
                    root.after(5, toggle_text_label)

        update_color(1)

toggle_text_label()  # 깜박임 시작

def free_brushing():
    global current_text_index, timer_running, blinking
    try:
        if blinking==False:
            text = sound_texts2[current_text_index]
            if text == "종료":
                text_label.config(text="\n"+text, fg="#BDE563", bg="#292E3B")
                timer_label.pack_forget()
            else:
                text_label.config(text="\n"+text, fg="#70F076", bg="#292E3B")
                # 타이머 설정
                for seconds in range(60, 0, -1):
                    if timer_running:
                        if seconds>10:
                            timer_label.config(text=f"남은 시간: {seconds} 초", bg="#292E3B", fg="white")
                        else:
                            timer_label.config(text=f"남은 시간: {seconds} 초", bg="#292E3B", fg="red")
                        root.update()
                        time.sleep(1)
                    else:
                        break  # 중지 요청이 들어오면 타이머 종료
                if timer_running:
                    print(current_text_index)
                    root.after(0, free_brushing)
                    pygame.mixer.music.load("beep.wav")
                    #pygame.mixer.music.play()
                else:
                    timer_label.config(text="타이머 중지", bg="#292E3B")
            current_text_index += 1
    except IndexError:
        text_label.config(text="종료, 감사합니다", fg="#70F076")

def area_brushing():
    global current_text_index, timer_running, blinking,label
    try:
        if blinking==False:
            text = sound_texts[current_text_index]
            if text == "종료":
                save_values()
                text_label.config("\n"+text, fg="#BDE563", bg="#292E3B")
                next_label.pack_forget()
                timer_label.pack_forget()
            else:
                print('ss')
                print(is_space)
                label=current_text_index
                next_text_index = current_text_index + 1
                next_text = sound_texts[next_text_index]
                next_label.pack()
                timer_label.pack()
                text_label.config(text="\n"+text, fg="#70F076", bg="#292E3B")
                next_label.config(text=f"\n다음구역: {next_text}", bg="#292E3B")
                # 타이머 설정
                if current_text_index==0:
                    for seconds in range(1, 0, -1):
                        if timer_running:
                            if seconds>=4:
                                timer_label.config(text=f"남은 시간: {seconds} 초", bg="#292E3B", fg="white")
                            else:
                                timer_label.config(text=f"남은 시간: {seconds} 초", bg="#292E3B", fg="red")
                            root.update()
                            time.sleep(1)
                        else:
                            break  # 중지 요청이 들어오면 타이머 종료
                else:
                    for seconds in range(4, 0, -1):
                        if timer_running:
                            if seconds>=4:
                                timer_label.config(text=f"남은 시간: {seconds} 초", bg="#292E3B", fg="white")
                            else:
                                timer_label.config(text=f"남은 시간: {seconds} 초", bg="#292E3B", fg="red")
                            root.update()
                            time.sleep(1)
                        else:
                            break  # 중지 요청이 들어오면 타이머 종료
                if timer_running:
                    print(current_text_index)
                    pygame.mixer.music.load("beep.wav")
                    #pygame.mixer.music.play()
                    next_label.pack_forget()
                    timer_label.pack_forget()
                    text_label.config(text=f"\n다음구역: {next_text}", fg="#70F076", bg="#292E3B") 
                    root.after(1500, area_brushing)
                else:
                    timer_label.config(text="타이머 중지", bg="#292E3B")
            current_text_index = next_text_index
    except IndexError:
        text_label.config(text="종료, 감사합니다", fg="#70F076")

def c_sequence():
    global current_text_index, timer_running
    current_text_index = 0
    timer_running = True  # 타이머 시작
    # Start the sequence
    free_brushing()

# Create a function to start the sequence
def space_sequence():
    global current_text_index, timer_running
    current_text_index = 0
    timer_running = True  # 타이머 시작
    # Start the sequence
    area_brushing()

def on_c_key(event):
    global timer_running, blinking,is_c
    print(is_space)
    if is_space!=True:
        root.configure(bg="#292E3B")
        text_label.config(fg="#70F076",font=font_70size)

        if timer_running:
            is_c=False
            timer_running = False  # 타이머 중지
            if(timer_label.winfo_ismapped):
                root.configure(bg="#F0F0F0")
                text_label.pack_forget()
                timer_label.pack_forget()
                title_frame.pack()
                subtitle_label.pack()
                press_kr_label.pack()
                text_label.pack()
                text_label.config(text="PRESS SPACE KEY", font=font_30size, anchor="center", bg=main_bg, fg="#479C2F")
                blinking = True
                toggle_text_label()
        else:
            is_c=True
            blinking = False
            title_frame.pack_forget()
            subtitle_label.pack_forget()
            press_kr_label.pack_forget()
            text_label.pack()
            timer_label.pack()
            c_sequence()

def on_space(event):
    print(is_c)
    global timer_running, blinking,is_space,msg,msg_label,msg_time
    if is_c!=True:
        root.configure(bg="#292E3B")
        text_label.config(fg="#70F076",font=font_70size)

        if timer_running:
            is_space=False
            timer_running = False  # 타이머 중지
            if(next_label.winfo_ismapped):
                root.configure(bg="#F0F0F0")
                text_label.pack_forget()
                next_label.pack_forget()
                timer_label.pack_forget()
                title_frame.pack()
                subtitle_label.pack()
                press_kr_label.pack()
                text_label.pack()
                text_label.config(text="PRESS SPACE KEY", font=font_30size, anchor="center", bg=main_bg, fg="#479C2F")
                blinking = True
                toggle_text_label()
        else:
            is_space=True
            blinking = False
            title_frame.pack_forget()
            subtitle_label.pack_forget()
            press_kr_label.pack_forget()
            text_label.pack()
            next_label.pack()
            timer_label.pack()
            msg=[]
            msg_label=[]
            msg_time=[]
            space_sequence()

root.bind("<space>", on_space)
root.bind("<c>", on_c_key)

root.mainloop()


Exception in thread Thread-19:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/pj/5xhy0s2d6bx728d6w_6bwp1w0000gn/T/ipykernel_74314/1875452342.py", line 157, in serial_communication
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/serial/serialutil.py", line 663, in read_until
    c = self.read(1)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/serial/serialposix.py", line 595, in read
    raise SerialException(
serial.serialutil.SerialException: device reports readiness to read but returned no data (device disconnected or multiple access on port?)


False
ss
True
0
ss
True
1
ss
True
2
ss
True
3
ss
True
4
ss
True
5
ss
True
6
ss
True
7
ss
True
8
ss
True
9
ss
True
10
ss
True
11
ss
True
12
ss
True
13
['[21:11:41.660843],[ 1.03271484 -0.06738281 -0.03417969],[ 317.74902344  -68.48144531 -116.08886719],[-1]', '[21:11:41.661423],[0.98876953 0.03076172 0.00927734],[ 297.54638672  -46.875      -120.84960938],[-1]', '[21:11:41.662015],[0.93652344 0.10839844 0.05810547],[ 253.17382812  -21.30126953 -102.47802734],[-1]', '[21:11:41.662604],[0.92822266 0.11328125 0.08251953],[213.01269531  -7.69042969 -73.05908203],[-1]', '[21:11:41.663167],[0.98046875 0.046875   0.10839844],[194.82421875  -3.29589844 -47.8515625 ],[-1]', '[21:11:41.663728],[ 1.03515625 -0.02001953  0.13574219],[187.98828125   7.50732422 -32.65380859],[-1]', '[21:11:41.664289],[ 1.06933594 -0.07568359  0.13525391],[179.01611328  28.93066406 -24.59716797],[-1]', '[21:11:41.664869],[ 1.08105469 -0.11816406  0.14941406],[167.48046875  49.74365234 -20.01953125],[-1]', '[21:11:41.6

Exception in Tkinter callback
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/tkinter/__init__.py", line 1892, in __call__
    return self.func(*args)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/tkinter/__init__.py", line 814, in callit
    func(*args)
  File "/var/folders/pj/5xhy0s2d6bx728d6w_6bwp1w0000gn/T/ipykernel_74314/88146181.py", line 234, in area_brushing
    text_label.config("\n"+text, fg="#BDE563", bg="#292E3B")
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/tkinter/__init__.py", line 1646, in configure
    return self._configure('configure', cnf, kw)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/tkinter/__init__.py", line 1629, in _configure
    cnf = _cnfmerge((cnf, kw))
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/tkinter/__init__.py", line 111, in _cnfmerge
    cnf.update(c)
ValueError: dictionary update seq

In [ ]:
sound_texts = [
    "앞니 바깥면",
    "오른쪽 바깥면",
    "왼쪽 바깥면",
    "오른쪽 아래 씹는면",
    "오른쪽 위 씹는면",
    "왼쪽 아래 씹는면",
    "왼쪽 위 씹는면",
    "오른쪽 위 안쪽면",
    "오른쪽 아래 안쪽면",
    "왼쪽 위 안쪽면",
    "왼쪽 아래 안쪽면",
    "앞니 위 안쪽면",
    "앞니 아래 안쪽면",
    "종료"
]